In [ ]:
import sys, os
repodir = os.path.abspath('..')
if not repodir in sys.path:
    sys.path.append(repodir)

In [ ]:
from aircleaning import svg, html

In [ ]:
out = html.Page('Foo', (svg.draw_scene(),))

In [ ]:
out._repr_html_()

In [ ]:
out

In [ ]:
from aircleaning.produce import *

In [ ]:



def decision_tool_with_room(path=productsdir, name='decision_tool'):

    vols, quals = load.get_volume_data(), load.get_quality_data()

    strn = ''

    strn += '\n' + '\n'.join((
        '''<link rel="stylesheet" href="//dds-gen3.web.unimelb.edu.au/v12.1.3/uom.css">''',
        '''<link rel="stylesheet" href="https://cms.unimelb.edu.au/__data/assets/css_file_folder/0008/3224951/uom-mce-gen3.css?v=0.0.202">''',
        ))

    strn += '\n' + '\n'.join((
        '''<script>''',
        '''    function update (form){''',
        '''        for (vol = 0; vol < 3; vol++) {''',
        '''            if (form.volume[vol].checked)''',
        '''                break;''',
        '''            }''',
        '''        for (qual = 0; qual < 3; qual++) {''',
        '''            if (form.quality[qual].checked)''',
        '''                break;''',
        '''            }''',
        '''        image = document.getElementById('chosenimage')   ''',
        '''        image.src = "https://rsbyrne.github.io/aircleaning/products/" + vol + "_" + qual + ".png"''',
        '''        image.alt = "Volume chosen: " + vol + " , quality chosen: " + qual''',
        '''        }''',
        '''</script>''',
        ))

    strn += '\n' + '\n'.join((
        '''<div class="uomcontent" role="document" id="top">''',
        # '''<h1>Decision Tool</h2>''',
        '''<div class="main" id="" role="main">''',
        '''<p class="notice notice--info" style="padding-top: 1.5rem;">''',
        '''Running costs and noise levels (on the highest fan settings) are calculated from manufacturer data. Select the approximate volume and the additional air cleaning to ventilation that you require. Natural ventilation provides 1-2 ACH; mechanical ventilation is higher. You will see that multiple devices sometimes provides the best effective air cleaning for the best value and noise requirements of your space.''',
        '''</p>''',
        '''<form id="userinput" >''',
        ))

    strn += '\n' + '\n'.join((
        '''<div class="sq-form-question sq-form-question-tickbox-list ">''',
        '''<fieldset id="volumeoptions" class="sq-form-section">''',
        '''<legend class="sq-form-section-title">How big is the room you want to put your air purifier in?</legend>''',
        ))
    strn += '\n' + make_cost_analysis_form_channel('volume', vols)
    strn += '\n' + '\n'.join((
        '''</fieldset>''',
        '''</div>'''
        ))

    strn += '\n' + '\n'.join((
        '''<div class="sq-form-question sq-form-question-tickbox-list ">''',
        '''<br/>''',
        '''<fieldset id="qualityoptions" class="sq-form-section">''',
        '''<legend class="sq-form-section-title">How much additional air cleaning do you require?</legend>''',
        ))
    strn += '\n' + make_cost_analysis_form_channel('quality', quals)
    strn += '\n' + '\n'.join((
        '''</fieldset>''',
        '''</div>'''
        ))

    strn += '\n' + '\n'.join((
        '''</form>''',
        f'''<p><em>Last updated {str(date.today())}</em><p>''',
        '''<figure class="figure figure--min">'''
        '''<img id = 'chosenimage' src='https://rsbyrne.github.io/aircleaning/products/placeholder.png' alt="Cost analysis">''',
        '''</figure>''',
        ))

    strn += '\n' + '\n'.join((
        '''<script>''',
        '''update(document.getElementById('userinput'))''',
        '''</script>''',
        ))

    with open(os.path.join(productsdir, name) + '.html', mode='w') as file:
        file.write(strn)